# Datatón BC 2018 - Robocoach (Asignacion de categoría de acuerdo a ref1)

Se parte generando una base de datos en formato SQL con las tablas suministradas, esta base de datos está en la platafor AZURE de Microsoft y será cargada desde Google Colab por medio de un notebook de Python

La máquina virtual de Google Colab requiere la instalación de los siguientes paquetes para cargar la base de datos SQL 

In [1]:
!apt-get install libqt4-sql-odbc
!apt-get install unixodbc unixodbc-dev --install-suggests
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/18.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install msodbcsql17

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libqt4-sql-odbc is already the newest version (4:4.8.7+dfsg-7ubuntu1).
The following packages were automatically installed and are no longer required:
  autoconf autoconf-archive autoconf-doc automake autopoint gettext
  gettext-base gettext-doc gnu-standards libasprintf-dev libasprintf0v5
  libcroco3 libgettextpo-dev libgettextpo0 libsigsegv2 libtool-doc m4 m4-doc
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unixodbc-dev is already the newest version (2.3.4-1.1ubuntu3).
unixodbc is already the newest version (2.3.4-1.1ubuntu3).
The following packages were automatically installed and are no longer required:
  autoconf autoconf-archive autoconf-doc automake autopoint gettext
  gettext-base gettext-doc gnu-standards libasprintf-dev libasprintf0v

In [2]:
 !pip install pyodbc

Carga de las librerías de Python utilizadas

In [0]:
import numpy as np
import pandas as pd
import pyodbc
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk import download
import string

Verificación de la instalación de pyodbc y del driver para gestionar SQL 

In [4]:
pyodbc.drivers()

['ODBC Driver 17 for SQL Server']

Definición de la conexión con el servidor de la base de datos de entrada

In [0]:
conn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=tcp:robocoach.database.windows.net,1433;" \
                      "Database=datathon_database;Uid=datathon_admin@robocoach;Pwd=robocoach.2018;Encrypt=yes;"\
                      "TrustServerCertificate=yes;Connection Timeout=120;encoding='cp1252'")

Carga de la primera vista de los datos, esta se hace con el objetivo de definir las categorías que serán etiquetas de los datos. 
Se cargan las ref1 de todos los datos (transacciones) 

In [0]:
sql = "SELECT id_trn_ach, ref1, id_categoria FROM transacciones_personas_v" # se demora 1061 s (18 min) segundos
#sql = "SELECT ref1 FROM transacciones_personas_v"


data = pd.read_sql(sql,conn)

#print(data.to_string())

In [7]:
data.to_csv('ref1.csv')
data

,id_trn_ach,ref1,id_categoria
0,338478281,Pago de factura Gas Natural CupÃ³n No,NaN
1,263608276,Pago de factura Gas Natural CupÃ³n No,NaN
2,273480411,TARJETA DE CREDITO VISA Y AMEX,NaN
3,301515656,Pago de factura Gas Natural CupÃ³n No,NaN
4,305083571,pagos,NaN
5,318454976,Pago de factura Gas Natural CupÃ³n No,NaN
6,259699620,Pago facturas: CMR,NaN
7,274243749,Pago facturas: CMR,NaN
8,231327537,Pago de Saldo,13.0
9,355900117,CARTERA,NaN


Número total de datos

In [8]:
np.size(data)

35561346

Crear diccionario para reemplazar caractéres con tildes antes del vectorizer
(http://www.merinomartinez.com/2012/06/tabla-de-conversion-de-simbolos-utf-8-a-iso-8859-1-ansi/)

In [9]:
accents_dict = {
    #"Ã?": "A", 
    "Ã‰":"E", 
    "Ã": "I", 
    "Ã“": "O", 
    "Ãš": "U",
    "Ã¡": "a",
    "Ã³": "o", 
    "Ã©": "e",  
    "Ãº": "u",
    "Ã­": "i",
    "Ã‘": "Ñ",
    "Ã±": "ñ",
}

data['ref1'].replace(to_replace=accents_dict, inplace=True, regex=True)
data

,id_trn_ach,ref1,id_categoria
0,338478281,Pago de factura Gas Natural Cupon No,NaN
1,263608276,Pago de factura Gas Natural Cupon No,NaN
2,273480411,TARJETA DE CREDITO VISA Y AMEX,NaN
3,301515656,Pago de factura Gas Natural Cupon No,NaN
4,305083571,pagos,NaN
5,318454976,Pago de factura Gas Natural Cupon No,NaN
6,259699620,Pago facturas: CMR,NaN
7,274243749,Pago facturas: CMR,NaN
8,231327537,Pago de Saldo,13.0
9,355900117,CARTERA,NaN


Cargar las reglas de negocio para etiquetar transacciones

In [11]:
ref1_rules = pd.read_csv('regla_ref1.csv', encoding ='latin1')
ref1_rules

,Palabra en ref1,Categoria
0,avianca,vajes
1,administracion,vivienda
2,administraci,vivienda
3,celular,Tecnología y comunicaciones
4,colegio,Educacion
5,apto,vivienda
6,flight,viajes
7,impuesto,Impuestos
8,impuestos,Impuestos
9,curso,Educacion


Se aplican las reglas de negocio a todas las transacciones

In [0]:
data['ref1'] = data['ref1'].astype(str)
ref1_rules['Palabra en ref1'] = ref1_rules['Palabra en ref1'].astype(str)
labels = np.array([])
count = 0;
count3 = 0;
for ref1 in data['ref1']:
  ref1 = ref1.lower()
  categ = data['id_categoria'][count]
  count2 = 0;
  for word in ref1_rules['Palabra en ref1']:
    #if word in ref1 and categ is None:
    if word in ref1:
      count3 = count3 + 1
      print(categ)
      labels.extend([count, count2])
    count2 = count2 + 1
count = count+1
  
labels